In [2]:
from supermechs.ext.deserializers.models import to_item_data
from supermechs.platform import json_decoder

with open("data/example_item_v2.json") as file:
    data = json_decoder(file.read())

item = to_item_data(data, "@Darkstare", False)

In [3]:
from supermechs.typeshed import dict_items_as
from supermechs.item_stats import AnyStatsMapping, TransformStage, ValueRange, lerp, lerp_vector


def at(stage: TransformStage, level: int, /) -> AnyStatsMapping:
    """Returns the stats at given level.

    For convenience, levels follow the game logic; the lowest level is 1
    and the maximum is a multiple of 10 depending on tier.
    """
    level -= 1

    if level == stage._last[0]:
        return stage._last[1]

    max_level = stage.tier.max_level

    if not 0 <= level <= max_level:
        raise ValueError(f"Level {level} outside range 1-{max_level+1}")

    if level == 0:
        return stage.base_stats.copy()

    if level == max_level:
        return stage.base_stats | stage.max_level_stats

    weight = level / max_level
    stats = stage.base_stats.copy()

    for key, value in dict_items_as(int | ValueRange, stage.max_level_stats):
        base_value: int | ValueRange = stats[key]

        if isinstance(value, ValueRange):
            assert isinstance(base_value, ValueRange)
            stats[key] = lerp_vector(base_value, value, weight)

        else:
            assert not isinstance(base_value, ValueRange)
            stats[key] = lerp(base_value, value, weight)

    stage._last = (level, stats.copy())
    return stats

print(item.start_stage.base_stats)
print(at(item.start_stage, item.start_stage.max_level))


{'weight': 63, 'expDmg': ValueRange(lower=213, upper=382), 'heaDmg': 112, 'expResDmg': 14, 'range': ValueRange(lower=3, upper=6), 'push': 1, 'retreat': 1, 'uses': 3, 'backfire': 180, 'heaCost': 75}
{}
